In [1]:
# !unzip robohearts.zip
#%cd robohearts/

In [1]:
import gym
import multiprocessing
from gymhearts.Hearts import *
from gymhearts.Agent.human import Human
from gymhearts.Agent.random_agent import RandomAgent
from gymhearts.Agent.monte_carlo_nn import MonteCarloNN
from gymhearts.Agent.agent_utils import save_model
from tqdm import tqdm_notebook

In [13]:
# ----------- TRAINING CONFIGURATION -------------
# Number of games to train on 
TRAINING_ITERS = 1000

# Number of episodes to run during model evaluation
NUM_EPISODES = 1000

# Number of model evaluations to average together
NUM_TESTS = 10

# Max score for players to win the game
MAX_SCORE = 100

# Set to false to skip training
run_train = True

# Set to true to resume training from an existing model
continue_train = False

# Features to include in model :: [in_hand, in_play, played_cards, won_cards, scores]
feature_list = ['in_hand', 'in_play']

# Name of the file that is saved :: {model_name}.th
model_name = 'mlp'

# Configuration parameters for the model
mc_config = {
    'print_info' : False,
    'epsilon' : .05,
    'gamma' : 1.0,
    'alpha': 1e-2,
    'feature_list' : feature_list
}

if continue_train:
    mc_config['load_model'] = model_name

playersNameList = ['MonteCarlo', 'Rando', 'Randy', 'Randall']
agent_list = [0, 0, 0, 0]

agent_list[0] = MonteCarloNN(playersNameList[0], mc_config)
agent_list[1] = RandomAgent(playersNameList[1], {'print_info' : False})
agent_list[2] = RandomAgent(playersNameList[2], {'print_info' : False})
agent_list[3] = RandomAgent(playersNameList[3], {'print_info' : False})

loaded from /home/akashkw/repos/robohearts/gymhearts/Agent/mlp_temp.th


In [8]:
# TRAIN THE MONTE CARLO AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
if run_train:
    for trn_episode in tqdm_notebook(range(TRAINING_ITERS)):
        # Save the model every 50 steps
        if trn_episode % 50 == 0:
            save_model(agent_list[0].nn, model_name)
        observation = env.reset()
        history = []
        while True:
            #env.render()

            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            # update my agent before clearing state
            if now_event == 'RoundEnd':
                agent_list[0].update_weights(history, -reward['MonteCarlo'])
                history = []
            if now_event == 'GameOver':
                  break
            if observation['event_name']=='PlayTrick' and observation['data']['playerName'] == 'MonteCarlo':
                # don't add score, they don't change till at end of round
                history.append((observation, agent_list[0].played_cards, agent_list[0].won_cards))

            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)

            observation, reward, done, info = env.step(action)
    save_model(agent_list[0].nn, model_name)

In [5]:
# Function to test model, meant to be passed into multiprocessing pool
def run_test(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'MonteCarlo')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [14]:
# EVALUATE THE MONTE CARLO AGENT
    
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)

# Evaluation parameters for testing
mc_config = {
    'print_info' : False,
    'load_model' : model_name,
    'feature_list' : feature_list
}
agent_list[0] = MonteCarloNN(playersNameList[0], mc_config)
mc_wins = [0] * NUM_TESTS        
pool = multiprocessing.Pool(processes=NUM_TESTS)
mc_wins = pool.map(run_test, mc_wins)
print(mc_wins)
pool.close()
pool.join()

loaded from /home/akashkw/repos/robohearts/gymhearts/Agent/mlp_temp.th
   




[281, 298, 309]


In [ ]:
def run_test_2(num_won):
    # Weird hack to make progress bars render properly
    print(' ', end='', flush=True)
    for i_ep in tqdm_notebook(range(NUM_EPISODES)):
        observation = env.reset()
        while True:
            now_event = observation['event_name']
            IsBroadcast = observation['broadcast']
            action = None
            if IsBroadcast == True:
                for agent in agent_list:
                    agent.Do_Action(observation)
            else:
                playName = observation['data']['playerName']
                for agent in agent_list:
                    if agent.name == playName:
                        action = agent.Do_Action(observation)
            if now_event == 'GameOver':
                num_won += int(observation['data']['Winner'] == 'Randman')
                break
            observation, reward, done, info = env.step(action)
    return num_won

In [ ]:
# EVALUATE THE RANDOM AGENT
env = gym.make('Hearts_Card_Game-v0')
env.__init__(playersNameList, MAX_SCORE)
playersNameList[0] = 'Randman'
agent_list[0] = RandomAgent(playersNameList[0])
rand_wins = [0] * NUM_TESTS
        
pool = multiprocessing.Pool(processes=NUM_TESTS)
rand_wins = pool.map(run_test_2, rand_wins)
print(rand_wins)
pool.close()
pool.join()

In [10]:
print(f"Monte Carlo won {sum(mc_wins)/len(mc_wins)} times on average :: {str(mc_wins)}")
for win in mc_wins:
    print(win)
#print(f"Random won {sum(rand_wins)/len(rand_wins)} times on average :: {str(rand_wins)}")

Monte Carlo won 293.5 times on average :: [279, 290, 315, 290, 283, 306, 326, 291, 277, 278]


NameError: name 'rand_wins' is not defined